# Create embeddings

In [12]:
import chromadb
from chromadb.utils import embedding_functions

cot_template_chromadb_path = "../chromadb/"
client = chromadb.PersistentClient(path=cot_template_chromadb_path)

In [13]:
collection = client.create_collection(
    name="cot_template",
)

In [14]:
import csv
from openai import OpenAI

openai_client = OpenAI()

with open(f'../data/cot-template/exam-72-result.csv', mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)
    exam_result = [row for row in reader]

In [15]:
COT_TEMPLATE = """
## Question: {question}
{answer_choices}
## Answer
{response}
"""

In [16]:
def add_collection(collection, exam_result):
    ids=[]
    metadatas=[]
    documents=[]
    embeddings=[]
    for item in exam_result:
        embedding = openai_client.embeddings.create(input = [item["question"]], model="text-embedding-ada-002").data[0].embedding
        document = COT_TEMPLATE.format(
            question=item["question"],
            answer_choices=item["answer_choices"],
            response=item["response"]
        )
        
        ids.append(f"72-{item['no']}")
        documents.append(document)
        metadatas.append({'exam': 72, 'question_no': item['no']})
        embeddings.append(embedding) 

    collection.add(
        documents=documents,
        embeddings=embeddings,
        metadatas=metadatas,
        ids=ids
    )

In [17]:
add_collection(collection, exam_result)

In [18]:
collection.count()

12